# naive Bayesian classifier

naive Bayes classifier is under attribute conditional independence assumption.

$$P(c|\mathbb{x})=\frac{P(c)}{P(\mathbb{x})}\prod^d_{i=1}P(\mathbb{x}_i|c)$$

use Laplacian correction:

$$\hat{P}(c) = \frac{|D_c|+1}{|D|+N} $$
$$\hat{P}(x_i|c) = \frac{|D_{c,x_i}|+1}{|D_c|+N_i}$$ 

# Problem 3

* problem7.3 :
 
试编程实现拉普拉斯修正的朴素贝叶斯分类器，并以西瓜数据集 3.0
为训练集，对 p.151 "测 1" 样本进行判别.

In [3]:
import pandas as pd
import numpy as np

# import data
data_melon = pd.read_csv("./dataset/西瓜数据集3.0.csv")
data_melon.head()

,编号,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率,好瓜
0,1,青绿,蜷缩,浊响,清晰,凹陷,硬滑,0.697,0.460,是
1,2,乌黑,蜷缩,沉闷,清晰,凹陷,硬滑,0.774,0.376,是
2,3,乌黑,蜷缩,浊响,清晰,凹陷,硬滑,0.634,0.264,是
3,4,青绿,蜷缩,沉闷,清晰,凹陷,硬滑,0.608,0.318,是
4,5,浅白,蜷缩,浊响,清晰,凹陷,硬滑,0.556,0.215,是


In [4]:
x_test = pd.DataFrame([["测1", "青绿", "蜷缩", "浊响", "清晰",
                    "凹陷", "硬滑", 0.697, 0.460]],
                    columns=data_melon.columns[:-1])
x_test

,编号,色泽,根蒂,敲声,纹理,脐部,触感,密度,含糖率
0,测1,青绿,蜷缩,浊响,清晰,凹陷,硬滑,0.697,0.46


In [5]:
from scipy.stats import norm

# MLE
# P(class)
P_c_x = np.ones(2)
c = np.unique(data_melon.loc[:, "好瓜"])
P_c_x *= np.sum((data_melon.loc[:, "好瓜"].to_numpy().reshape(-1,1) == c).astype(int),
                axis=0)/data_melon.shape[0]
# P(xi|class)
for i in range(len(c)):
    data_c = data_melon[data_melon.loc[:, "好瓜"] == c[i]]
    for xi_idx in x_test.iloc[0, 1:].index:
        xi = x_test.iloc[0, :].loc[xi_idx]
        if isinstance(xi, str):
            P_c_x[i] *= (np.sum((data_c.loc[:, xi_idx] == xi).astype(int)) + 1) \
                / (data_c.shape[0] + np.unique(data_c.loc[:, xi_idx].to_numpy()).shape[0])
        elif isinstance(xi, float):
            Di_c = data_c.loc[:, xi_idx].to_numpy()
            P_c_x[i] *= norm.pdf(xi, loc=np.mean(Di_c), scale=np.std(Di_c, ddof=1)) # ddof=1 is unbiased std
        else:
            raise ValueError("something wrong")
# normalization
P_c_x = P_c_x / np.sum(P_c_x)
print(f"posterior probability of class {c}: {P_c_x}")

posterior probability of class ['否' '是']: [0.00207924 0.99792076]


# Problem 10

**Problem:**

以西瓜数据集 2.0 中属性"脐部"为隐变量，试基于 EM 算法构建一
个贝叶斯网.

**Solution:**

构建流程：

1. 首先确定observation和latent variable，即确定好节点
2. 根据BIC准则预先得到一个Bayesian network的结构
3. 通过EM算法得到上面的Markov chain的条件概率

具体实现就太麻烦了，不写了